# Memory Optimization for HR Analytics Dataset

**Executive Summary:**
In this project, I performed data type optimization and memory footprint reduction on a customer dataset. By strictly defining data types (downcasting numerics, mapping booleans, and structuring ordinal/nominal categories), I successfully reduced the dataframe's memory usage by **over 96.5%** (from 2.0+ MB to ~69.5 KB). This optimization is a crucial step in MLOps and Data Engineering to ensure scalable, faster data loading, and efficient model training pipelines.

---

## 1. The Business Problem

A common bottleneck when deploying machine learning models to generate business value is the sheer volume of data. Inefficiently stored datasets can significantly slow down I/O operations and model training.

*Training Data Ltd.*, an online data science training provider, needs to clean up one of their largest customer datasets. This dataset will eventually feed a predictive model to determine if students are looking for a new job. 

To create a proof-of-concept for a more efficient storage solution, I was tasked with cleaning and optimizing the `customer_train.csv` subset containing anonymized student information.

**Data Dictionary:**

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
# Load the dataset
ds_jobs = pd.read_csv("datasets/customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


## 2. Establishing a Performance Baseline

Before applying any transformations, I will create a copy of the original dataframe. This allows us to benchmark the initial memory usage and validate the efficiency of our data type optimization strategy at the end of the process.

In [206]:
ds_jobs_transformed = ds_jobs.copy()

ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [207]:
print(ds_jobs.columns)

Index(['student_id', 'city', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'job_change'],
      dtype='object')


## 3. Data Type Optimization

### 3.1. Binary Features to Booleans
By default, Pandas stores text as `object` types, which allocate memory for pointers to arbitrary Python objects. Columns containing only binary states (e.g., `job_change` and `relevant_experience`) can be explicitly mapped to 1-bit `bool` arrays, drastically reducing their memory footprint.

In [208]:
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype(bool)
ds_jobs_transformed['relevant_experience'] = (ds_jobs_transformed['relevant_experience'] == 'Has relevant experience')

In [209]:
print('---- Original Dataframe ----')
print(ds_jobs[['job_change','relevant_experience' ]].info(),'\n')
print('---- Transformed Dataframe ----')
print(ds_jobs_transformed[['job_change','relevant_experience' ]].info(),'\n')

---- Original Dataframe ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   job_change           19158 non-null  float64
 1   relevant_experience  19158 non-null  object 
dtypes: float64(1), object(1)
memory usage: 299.5+ KB
None 

---- Transformed Dataframe ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   job_change           19158 non-null  bool 
 1   relevant_experience  19158 non-null  bool 
dtypes: bool(2)
memory usage: 37.5 KB
None 



As we can see, this reduces memory of these columns usage by approximately 87%.

### 3.2. Downcasting Numerical Variables
Pandas natively allocates numerical values to 64-bit integers (`int64`) or floats (`float64`). However, analyzing the maximum and minimum limits of our data:
- Features like `student_id` and `training_hours` fit comfortably within 32-bit limits.
- `city_development_index` requires less precision and can be safely stored as a 16-bit float.

Downcasting these arrays cuts the numerical memory allocation by half (or more) without any loss of information.

In [210]:
ds_jobs_transformed['student_id']  = ds_jobs_transformed['student_id'].astype('int32')
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')
ds_jobs_transformed['city_development_index']  = ds_jobs_transformed['city_development_index'].astype('float16')

In [211]:
print('---- Original Dataframe ----')
print(ds_jobs[['student_id','training_hours','city_development_index' ]].info(),'\n')
print('---- Transformed Dataframe ----')
print(ds_jobs_transformed[['student_id','training_hours','city_development_index' ]].info(),'\n') 

---- Original Dataframe ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   training_hours          19158 non-null  int64  
 2   city_development_index  19158 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 449.1 KB
None 

---- Transformed Dataframe ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int32  
 1   training_hours          19158 non-null  int32  
 2   city_development_index  19158 non-null  float16
dtypes: float16(1), int32(2)
memory usage: 187.2 KB
None 



This change reduced the memory usage by 58%.

### 3.3. Structuring Nominal and Ordinal Categories
Storing repetitive text data as `object` types is highly inefficient. Converting them to the `category` data type maps each unique string to an integer array under the hood, saving massive amounts of RAM.

Furthermore, applying an **ordinal structure** to variables like `experience`, `education_level`, and `company_size` is not just about memory—it ensures that any future Machine Learning algorithms inherently understand the hierarchical relationship of these features (e.g., Masters > Graduate).


In [212]:
print(ds_jobs_transformed['gender'].value_counts())
print(ds_jobs_transformed['city'].value_counts())
print(ds_jobs_transformed['major_discipline'].value_counts())
print(ds_jobs_transformed['company_type'].value_counts())
print(ds_jobs_transformed['education_level'].value_counts())
print(ds_jobs_transformed['enrolled_university'].value_counts())
print(ds_jobs_transformed['experience'].value_counts())
print(ds_jobs_transformed['last_new_job'].value_counts())
print(ds_jobs_transformed['company_size'].value_counts())

Male      13221
Female     1238
Other       191
Name: gender, dtype: int64
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_129       3
city_111       3
city_121       3
city_140       1
city_171       1
Name: city, Length: 123, dtype: int64
STEM               14492
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: major_discipline, dtype: int64
Pvt Ltd                9817
Funded Startup         1001
Public Sector           955
Early Stage Startup     603
NGO                     521
Other                   121
Name: company_type, dtype: int64
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: education_level, dtype: int64
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: enrolled_university, dtype: int64
>20    3286
5      1430
4      1403
3 

In [213]:
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype('category')
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category')
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category')
ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype('category')

In [214]:
print('---- Original Dataframe ----')
print(ds_jobs[['gender','major_discipline','company_type','city']].info(),'\n')
print('---- Transformed Dataframe ----')
print(ds_jobs_transformed[['gender','enrolled_university','major_discipline','company_type','city']].info(),'\n') 

---- Original Dataframe ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   gender            14650 non-null  object
 1   major_discipline  16345 non-null  object
 2   company_type      13018 non-null  object
 3   city              19158 non-null  object
dtypes: object(4)
memory usage: 598.8+ KB
None 

---- Transformed Dataframe ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   gender               14650 non-null  category
 1   enrolled_university  18772 non-null  object  
 2   major_discipline     16345 non-null  category
 3   company_type         13018 non-null  category
 4   city                 19158 non-null  category
dtypes: category(4), object(1)
memory usa

In [215]:
last_new_job_cat = ['never', '1', '2', '3', '4', '>4']
ds_jobs_transformed['last_new_job']  =  pd.Categorical( ds_jobs_transformed['last_new_job'], categories= last_new_job_cat,ordered=True)

exp_cat = ['<1', '1', '2', '3', '4', '5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','>20']
ds_jobs_transformed['experience']  = pd.Categorical( ds_jobs_transformed['experience'], categories= exp_cat,ordered=True)

education_cat = ['Primary School','High School','Graduate','Masters','Phd']
ds_jobs_transformed['education_level']  = pd.Categorical( ds_jobs_transformed['education_level'], categories= education_cat,ordered=True)

size_cat = [
    '<10',
    '10-49',
    '50-99',
    '100-499',
    '500-999',
    '1000-4999',
    '5000-9999',
    '10000+'
]
ds_jobs_transformed['company_size']  = pd.Categorical( ds_jobs_transformed['company_size'], categories= size_cat,ordered=True)

univ_cat = ['no_enrollment', 'Part time course', 'Full time course']
ds_jobs_transformed['enrolled_university']  = pd.Categorical( ds_jobs_transformed['enrolled_university'], categories= univ_cat,ordered=True)

In [216]:
print('---- Original Dataframe ----')
print(ds_jobs[['last_new_job','experience','company_size','education_level','enrolled_university' ]].info(),'\n')
print('---- Transformed Dataframe ----')
print(ds_jobs_transformed[['last_new_job','experience','company_size','education_level','enrolled_university' ]].info(),'\n') 

---- Original Dataframe ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   last_new_job         18735 non-null  object
 1   experience           19093 non-null  object
 2   company_size         13220 non-null  object
 3   education_level      18698 non-null  object
 4   enrolled_university  18772 non-null  object
dtypes: object(5)
memory usage: 748.5+ KB
None 

---- Transformed Dataframe ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   last_new_job         18735 non-null  category
 1   experience           19093 non-null  category
 2   company_size         13220 non-null  category
 3   education_level      18698 non-null  category
 4   enrolled_university 

This adjustments reduced the memory usage by approximately 87%.

## 4. Applying Business Rules (Data Filtering)
With the dataset thoroughly optimized, we can now apply the specific business criteria requested by the HR team: targeting highly experienced candidates. We will filter the data to include only students with **10+ years of experience** working in companies with **at least 1,000 employees**.
---

In [217]:
filter = ( (ds_jobs_transformed['experience'] >='10') & (ds_jobs_transformed['company_size'] >='1000-4999'))
ds_jobs_transformed = ds_jobs_transformed[filter]



## 5. Conclusion & Impact

In [218]:
print('---- Dataframe Before the adjustments ----')
print(ds_jobs.info(),'\n')

print('---- Dataframe After the adjustments ----')
print(ds_jobs_transformed.info(),'\n')

---- Dataframe Before the adjustments ----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64


**Final Results:**
Through strict data typing and filtering, the memory usage dropped from **2.0+ MB to 69.5 KB**. 

In a real-world scenario, processing a dataset scaled to millions of rows using these optimizations means the difference between a pipeline crashing due to Out-Of-Memory (OOM) errors and running smoothly on a lightweight cloud instance (AWS/Docker). This highlights the critical importance of memory profiling prior to the model training phase.